In [28]:
list.files(".")

[1] "Covid-19 data.csv" "EDA-COVID.ipynb"

In [31]:
AllCovidData <- read.csv("Covid-19 data.csv", encoding="UTF-8")

In [32]:
AllCovidData

Date,Confirmed.COVID.19.cases..Total.,COVID.19.infection..Total.,Number.tested.for.COVID.19..Total.,Deaths.with.COVID.19..Total.,Deaths.with.COVID.19.per.day,Admitted.to.the.hospitals.due.to.COVID.19,Admitted.to.the.Intensive.Care.Unit.due.to.COVID.19,Patients.in.respirator.due.to.COVID.19
M03D21,1326,null,11657,22,3,206,42,35
M03D22,1395,null,12351,25,3,232,46,40
M03D23,1450,null,12843,31,6,254,55,47
M03D24,1577,null,13756,37,6,301,59,58
M03D25,1715,null,14870,43,6,350,87,76
M03D26,1851,null,15981,55,12,386,94,78
M03D27,2010,null,17275,67,12,430,109,89
M03D28,2201,null,18810,76,9,459,121,104
M03D29,2395,null,20198,84,8,499,131,113
M03D30,2555,null,21378,95,11,533,137,119


#### Check the data types of the loaded data

Here, we check the data types of the data loaded onto R. Since we load the data from a CSV file, R does not know what type of data certain inputs are. By default, it categorises unknown data types as Factor as they can store both strings and integers.

In [27]:
str(AllCovidData)

'data.frame':	249 obs. of  9 variables:
 $ Date                                       : Factor w/ 249 levels "M03D21","M03D22",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ Bekræftede.COVID.19.tilfælde..Total.       : Factor w/ 233 levels "10083","10218",..: 65 76 82 91 107 115 121 126 131 134 ...
 $ Overstået.COVID.19.infektion..Total.       : Factor w/ 221 levels "10044","10106",..: 221 221 221 221 221 221 221 221 221 221 ...
 $ Antal.testede.for.COVID.19..Total.         : Factor w/ 233 levels "1001512","100543",..: 14 20 23 30 38 45 53 63 70 76 ...
 $ Dødsfald.med.COVID.19..Total.              : int  22 25 31 37 43 55 67 76 84 95 ...
 $ Dødsfald.med.COVID.19.pr..dag              : int  3 3 6 6 6 12 12 9 8 11 ...
 $ Indlagte.på.sygehusene.ifm..COVID.19       : Factor w/ 148 levels "100","101","102",..: 56 64 75 87 99 106 113 117 120 128 ...
 $ Indlagte.på.Intensiv.afdeling.ifm..COVID.19: Factor w/ 81 levels "0","1","10","100",..: 51 55 58 60 75 80 7 12 15 16 ...
 $ Indlagte.i.respirator.ifm..COVID